<a href="https://colab.research.google.com/github/arsalanyaghoobi/Text_Embedding-GlobalAveragePooling-/blob/master/Text_Embedding_Glob_Average_Pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf 
print(tf.__version__)

2.2.0


In [ ]:
import tensorflow_datasets as tfds
imdb,info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
import numpy as np
training_data,test_data=imdb['train'],imdb['test']
training_sentences=[]
training_labels=[]
test_sentences=[]
test_labels=[]

In [ ]:
for s,l in training_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

In [ ]:
training_labels_final=np.array(training_labels)
test_labels_final=np.array(test_labels)

In [ ]:
vocab_size=10000
embeding_dim=16
max_length=120
oov_token="oov_token"
truncating='post'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=truncating)

In [ ]:
tokenizer.fit_on_texts(test_sentences)
word_index=tokenizer.word_index
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded=pad_sequences(sequences,maxlen=max_length,truncating=truncating)

In [ ]:
import tensorflow as tf
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(vocab_size,embeding_dim,input_length=max_length),
                           tf.keras.layers.GlobalAveragePooling1D(),
                           tf.keras.layers.Dense(50,activation='relu'),
                           tf.keras.layers.Dense(40,activation='relu'),
                           tf.keras.layers.Dense(20,activation='relu'),
                           tf.keras.layers.Dense(10,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')
                            ])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 50)                850       
_________________________________________________________________
dense_46 (Dense)             (None, 40)                2040      
_________________________________________________________________
dense_47 (Dense)             (None, 20)                820       
_________________________________________________________________
dense_48 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_49 (Dense)             (None, 1)               

In [ ]:
num_epochs=10
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(test_padded,test_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4540 - acc: 0.7734 - val_loss: 1.3367 - val_acc: 0.4997
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2815 - acc: 0.8866 - val_loss: 1.4420 - val_acc: 0.5004
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2312 - acc: 0.9126 - val_loss: 1.5970 - val_acc: 0.4994
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1965 - acc: 0.9270 - val_loss: 1.6931 - val_acc: 0.5004
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1734 - acc: 0.9395 - val_loss: 1.7430 - val_acc: 0.4996
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1547 - acc: 0.9498 - val_loss: 1.8141 - val_acc: 0.5008
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1398 - acc: 0.9553 - val_loss: 1.9879 - val_acc: 0.5004
Epoch 8/10
782/782 [==============================] - 5s 6ms/step - loss: 0.

In [ ]:
x=0
e=model.layers[x]
weights=e.get_weights()[x]
print(weights.shape)

(10000, 16)


In [ ]:
reverse_word_index=dict([(value,key)for(key,value)in word_index.items()])

In [ ]:
import io


out_v=io.open('vecs.tsv','w',encoding='utf-8')
out_m=io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+"\n")
  out_v.write("\t".join([str(x) for x in embeddings])+"\n")
out_m.close()
out_v.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>